# STEP 4.1: Modeling Random Forest

## Description of the methodology
> * Finalize bining of Target Variable
* Create Train and Test datasets
* Create a Random Forest pipeline
* Define key parameters
* Run the model on sub-train data set and test accuracy on the validation data set
* Select 2 mots accurate models based on the hyper-parameters, run it to get the confusion matrix
* Select Best RF model candidate and apply it to the main train/test dataset

## Import libraries

In [48]:
import numpy as np
import pandas as pd
import seaborn as sns
from collections import OrderedDict as OrderedDict

%matplotlib inline
import matplotlib.pyplot as plt

import re
from sklearn.preprocessing import Normalizer
import os
from sklearn import preprocessing
from scipy.stats import kurtosis
from scipy.stats import skew
from scipy import stats
from sklearn.preprocessing import power_transform
from sklearn.preprocessing import KBinsDiscretizer

from sklearn.ensemble import RandomForestClassifier
from sklearn.cluster import KMeans
from sklearn.naive_bayes import ComplementNB, MultinomialNB

from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import NearestNeighbors
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import ParameterGrid

from sklearn.linear_model import SGDClassifier

from sklearn.model_selection import KFold
from sklearn.model_selection import cross_validate
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegressionCV
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

from sklearn.tree import DecisionTreeClassifier

from sklearn.svm import LinearSVC
from sklearn.svm import SVC

from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report
from sklearn.metrics import roc_curve
import scikitplot as skplt

# t_NSE dimensionality reduction
from sklearn.manifold import TSNE

import random
from sklearn import ensemble

from sklearn.model_selection import StratifiedShuffleSplit

import warnings

with warnings.catch_warnings():
    warnings.simplefilter("ignore")


# Activate Seaborn style
sns.set()

## Load the file for analysis

In [2]:
# Importing the file and creating a dataframe
master_modeling = pd.read_csv(
    "C:/Users/fbaff/EPFL ML Python/5. Captsone/master_modeling.csv",
    low_memory=False,
    skipinitialspace=True,
)

In [3]:
# display all columns
pd.set_option("display.max_columns", None)

In [4]:
# remove the Unnamed column
master_modeling.drop("Unnamed: 0", axis=1, inplace=True)
master_modeling.shape

(194484, 351)

In [5]:
# Create a dataframe for the modeling phase (without text and not relevant features)
df_modeling = master_modeling.drop(["Title", "Post_ID", "Snippet"], axis=1)

In [6]:
df_modeling.shape

(194484, 348)

## Definition of # of classes for the Target Variable 'All_Impact'

> * We will split the variable in 3 classes using Scikit Learn preprocessing function KBinDiscretizer with the following parameters: number of bins 3, encode: ordinal and strategy: quantile
* Oridinal has been selected as we are trying to model a hierarchy between low and high tweet impact
* Quantile implies an even number of data points per class which would shape the model to learn about features for each class equally (avoiding unbalance classes)
* We may reconsider some of the value of the parameters depending on the modeling results

In [7]:
ai_bin = master_modeling[["ALL_Impact"]]

In [8]:
# Process binizer
est = KBinsDiscretizer(n_bins=3, encode="ordinal", strategy="quantile")
est.fit(ai_bin)
new_ai = est.transform(ai_bin)

In [9]:
# Call the edge of the different 3 bins
est.bin_edges_[0]

array([ 0., 30., 41., 80.])

In [10]:
new_ai_df = pd.DataFrame(new_ai)

In [11]:
new_ai_df.shape

(194484, 1)

In [12]:
df_modeling["All_impact bin"] = new_ai_df

In [13]:
df_modeling["All_impact bin"].value_counts()

2.0    69658
1.0    63049
0.0    61777
Name: All_impact bin, dtype: int64

In [14]:
# Remove the original All_Impact feature
df_modeling2 = df_modeling.drop(
    ["ALL_Impact", "TW_Hashtags", "ALL_Author", "TW_Account_Name"], axis=1
)

In [15]:
# Transform new All Impact feature type into int64
df_modeling2["All_impact bin"] = df_modeling2["All_impact bin"].astype(np.int64)

In [16]:
df_modeling2.shape

(194484, 345)

In [17]:
df_modeling.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,Sentiment,Company_Clarivate,Company_Informa,Company_Pearson,Company_RELX Group,Company_Thomson Reuters,Company_Wolters Kluwer,Country 2_Argentina,Country 2_Australia,Country 2_Belgium,Country 2_Brazil,Country 2_Canada,Country 2_Ecuador,Country 2_France,Country 2_Germany,Country 2_Hong Kong,Country 2_India,Country 2_Italy,Country 2_Japan,Country 2_Mexico,Country 2_Netherlands,Country 2_Other,Country 2_Philippines,Country 2_Russia,Country 2_Serbia,Country 2_Singapore,Country 2_South Africa,Country 2_Spain,Country 2_Switzerland,Country 2_United Arab Emirates,Country 2_United Kingdom,Country 2_United States,Country 2_Venezuela,ALL_Thread_Entry_Type_post,ALL_Thread_Entry_Type_reply,ALL_Thread_Entry_Type_share,TW_Account_Type_Not identified,TW_Account_Type_individual,TW_Account_Type_organisational,ALL_Impact,Log_TW_KredOutreach,Log_Nbreach,Log_TW_NbFollowers,Log_TW_NbFollowing,Log_TW_NbTweets,TW_Hashtags,ALL_Author,TW_Account_Name,All_impact bin
0,-0.043355,-0.026154,0.056330,0.096232,-0.079631,-0.052836,0.048258,-0.210815,0.129344,0.028728,-0.051351,-0.202983,-0.097575,-0.069580,0.023331,0.098674,-0.010539,0.010712,-0.056834,-0.094849,-0.026164,-0.002116,0.283895,-0.005615,-0.053263,0.018941,-0.010457,0.144206,0.044637,-0.004079,0.004364,0.031087,-0.100098,-0.137032,-0.058838,-0.054138,-0.043732,0.082092,0.078893,-0.026632,-0.063639,-0.104980,0.150564,0.134542,0.175049,-0.071198,-0.067627,-0.100016,-0.217041,0.096486,-0.092585,0.039103,0.064535,0.061666,0.087718,-0.012533,-0.117382,-0.018209,0.045420,-0.249552,0.033651,-0.044680,-0.068578,-0.040140,-0.078196,-0.103747,-0.085938,-0.124268,0.084961,0.139425,-0.037394,0.049886,-0.062134,-0.060832,-0.164551,0.026286,-0.033651,0.038818,-0.057373,0.061015,-0.099528,0.092122,0.036601,-0.060507,0.082275,-0.030721,-0.009196,0.103404,0.001943,0.038981,0.123454,0.011963,-0.078763,-0.101461,-0.139242,0.086426,-0.044942,-0.051346,0.214478,0.051982,-0.033285,0.013041,0.002767,0.002828,0.073032,0.163574,-0.208537,0.020345,0.135742,-0.006714,-0.044881,-0.061890,0.051839,-0.026937,0.045980,0.052348,-0.023600,-0.276123,0.043335,0.084157,0.105428,-0.003113,-0.206991,0.110575,0.118327,-0.052622,-0.067342,0.039185,0.058289,-0.039581,-0.196574,0.026133,0.011617,0.023051,-0.109202,0.043915,-0.011190,-0.012197,0.048096,0.011353,0.288900,-0.083293,-0.062876,-0.053426,0.064270,-0.034261,-0.005025,-0.016693,-0.088582,-0.205322,0.116984,-0.101969,-0.003977,-0.069417,0.023163,-0.142741,-0.004181,-0.026530,0.065626,-0.038544,-0.201538,0.103404,-0.099365,0.042969,-0.020793,-0.140340,0.140372,-0.044189,-0.116414,0.194603,-0.135277,0.025309,0.029427,-0.008776,0.021322,-0.011882,0.162964,-0.080648,0.194173,-0.153809,-0.049357,-0.024295,-0.039917,-0.067261,0.164591,0.001709,-0.085144,0.081889,0.068522,0.031911,0.046265,0.115234,-0.217855,-0.105306,0.009359,0.091675,-0.076823,0.013916,-0.052916,-0.074188,-0.033691,0.140869,-0.030874,-0.009588,0.01

## Create a train, validation and test datasets (from the main Train set of data)
> * I am facing a lack of computing resources (laptop with i7 Intel chip and 16 Go RAM, no GPU) which implies a very long time for training models, especially with the tuning of hyper-parameters. As a consequence, I have combined my computing resources with Google Colaboratory in order to tune several parameters in parallel.
* **The overall dataset is divided in 3 buckets:**
* Bucket 1 (train/test): split for training the Best Selected model (in case of more important computing resources)
* Bucket 2 (train1/valid1): split for training the Best model candidate of a given class (no cross-validation)
* Bucket 3 (train2/valid2): split for hyper-parameter tuning leading to select the Best model candidate (cross-validation maybe considered in some cases)
* We could limit the risk of overfitting by using a cross-validation approach. However, we may run the risk of very demanding computing resources as we will combine hyper-parameter optimization (GridSearch) and large dataset (194484 rows x 344 variables).
* A compromised approach would be to use the standard train/test dataset split and leverage cross-validation for the validation phase in the process for selecting the best model.

### Create X and y arrays

In [18]:
# Create an array from df_modeling2 excluding the target variable All impact bin
X = df_modeling2.drop(["All_impact bin"], axis=1)
X = np.array(X)
X.shape

(194484, 344)

In [19]:
# Create y array for the target variable All impact bin
y = df_modeling2["All_impact bin"]
y = np.array(y)
y.shape

(194484,)

In [20]:
# Convert the type of the input matrix to float
X = X.astype(np.float)

# Create train set
X_tr_main, X_test, y_tr_main, y_test = train_test_split(
    X, y, test_size=0.2, random_state=0
)

# Create validation and test sets for best model selected for a given class
X_tr_2nd, X_valid1, y_tr_2nd, y_valid1 = train_test_split(
    X_tr_main, y_tr_main, test_size=4000, train_size=15000, random_state=0
)

# Create validation and test sets for hyper-parameter tuning and selection of the best model candidate
X_tr_3rd, X_valid2, y_tr_3rd, y_valid2 = train_test_split(
    X_tr_2nd, y_tr_2nd, test_size=1500, train_size=5000, random_state=0
)

print("Train:", X_tr_main.shape, y_tr_main.shape)
print("Test:", X_test.shape, y_test.shape)
print("Train1:", X_tr_2nd.shape, y_tr_2nd.shape)
print("Valid1:", X_valid1.shape, y_valid1.shape)
print("Train2:", X_tr_3rd.shape, y_tr_3rd.shape)
print("Valid2:", X_valid2.shape, y_valid2.shape)

Train: (155587, 344) (155587,)
Test: (38897, 344) (38897,)
Train1: (15000, 344) (15000,)
Valid1: (4000, 344) (4000,)
Train2: (5000, 344) (5000,)
Valid2: (1500, 344) (1500,)


In [21]:
pd.value_counts(y_valid2, normalize=True)

2    0.364667
0    0.324000
1    0.311333
dtype: float64

### Create a Random Forest pipeline

In [22]:
# Create Random Forest pipeline
pipe_rf = Pipeline(
    [
        ("scaler", StandardScaler()),  # with standardization StandardScaler()
        (
            "PCA",
            PCA(n_components=200),
        ),  # 200 components to explain 95% of the variance (see first part of this notebook)
        ("rf", RandomForestClassifier(n_jobs=-1, random_state=0)),
    ]
)

In [23]:
# Get parameters
# pipe_rf.get_params()

### Define the grid of parameters
> * I have tested 2 sets of hyper-parameters (6 parameters, version underneath) and 4 hyperparameters (indicated in the section where the confusion matrix is performed)

In [24]:
# Grid of parameters
grid_rf = ParameterGrid(
    {
        "rf__n_estimators": [2000],  # of decision trees
        "rf__criterion": ["gini"],  # quality of split
        "rf__max_features": [
            0.5,
            0.8,
        ],  # % of features vs total number of features for looking at the best split
        "rf__max_depth": [5, 10],  # depth of trees
        "rf__min_samples_leaf": [
            2,
            4,
        ],  # # Minimum number of samples required at each leaf node
        "rf__bootstrap": [
            False
        ],  # Method of selecting samples for training each tree ('True' tested but do not improve the performance)
    }
)

# Print the number of combinations
print("Number of combinations:", len(grid_rf))

Number of combinations: 8


### Run the model on on sub-train data set (5 000 tweets) and test accuracy on the validation data set (1 500 tweets)

In [25]:
#  Save accuracy on train and validation sets
train_scores = []
valid_scores = []

# Enumerate combinations starting from 1
for i, params_dict in enumerate(grid_rf, 1):
    # Print progress
    print("Combination {}/{}".format(i, len(grid_rf)))  # Total number of combinations

    # Set parameters
    pipe_rf.set_params(**params_dict)

    # Fit a Decision Tree classifier
    pipe_rf.fit(X_tr_3rd, y_tr_3rd)

    # Save accuracy on validation set
    params_dict["accuracy_train"] = pipe_rf.score(X_tr_3rd, y_tr_3rd)
    params_dict["accuracy_valid"] = pipe_rf.score(X_valid2, y_valid2)

    # Save result
    train_scores.append(params_dict)
    valid_scores.append(params_dict)

print("done")

Combination 1/8
Combination 2/8
Combination 3/8
Combination 4/8
Combination 5/8
Combination 6/8
Combination 7/8
Combination 8/8
done


In [26]:
# Create DataFrame with test scores
scores_df = pd.DataFrame(valid_scores)
# Print scores
scores_df.sort_values(by="accuracy_valid", ascending=False)

,rf__bootstrap,rf__criterion,rf__max_depth,rf__max_features,rf__min_samples_leaf,rf__n_estimators,accuracy_train,accuracy_valid
5,False,gini,10,0.5,4,2000,0.9692,0.732667
4,False,gini,10,0.5,2,2000,0.9812,0.731333
0,False,gini,5,0.5,2,2000,0.7348,0.716667
1,False,gini,5,0.5,4,2000,0.7336,0.716000
6,False,gini,10,0.8,2,2000,0.9676,0.708000
7,False,gini,10,0.8,4,2000,0.9612,0.708000
2,False,gini,5,0.8,2,2000,0.7278,0.706000
3,False,gini,5,0.8,4,2000,0.7268,0.706000


### Side notes
> * I have tested several combination of hyper-parameters separatly due to the limitation of computing resources (with different size of train/valid datasets, from 5000 to 15000 for train dataset)
* It appears that the accuracy rates do not vary much and there is in any case some overfitting

In [27]:
# accuracy_train	accuracy_valid	rf__bootstrap	rf__criterion	rf__max_depth	rf__max_features	rf__min_samples_leaf	rf__n_estimators
# 2	0.9810	0.672000	True	gini	10	0.8	2	500
# 0	0.9826	0.669333	True	gini	10	0.5	2	500
# 1	0.9716	0.669333	True	gini	10	0.5	4	500
# 3	0.9718	0.658000	True	gini	10	0.8	4	500
# 4	0.9890	0.656000	False	gini	10	0.5	2	500
# 5	0.9862	0.656000	False	gini	10	0.5	4	500
# 6	0.9768	0.624667	False	gini	10	0.8	2	500
# 7	0.9704	0.622000	False	gini	10	0.8	4	500

In [28]:
# accuracy_train	accuracy_valid	rf__bootstrap	rf__criterion	rf__max_depth	rf__max_features	rf__min_samples_leaf	rf__n_estimators
# 2	0.9810	0.672000	True	gini	10	0.8	2	500
# 0	0.9826	0.669333	True	gini	10	0.5	2	500
# 1	0.9716	0.669333	True	gini	10	0.5	4	500
# 3	0.9718	0.658000	True	gini	10	0.8	4	500
# 4	0.9890	0.656000	False	gini	10	0.5	2	500
# 5	0.9862	0.656000	False	gini	10	0.5	4	500
# 6	0.9768	0.624667	False	gini	10	0.8	2	500
# 7	0.9704	0.622000	False	gini	10	0.8	4	500

In [29]:
# accuracy_train	accuracy_valid	rf__criterion	rf__max_depth	rf__max_features	rf__n_estimators
# 1	0.9672	0.698000	gini	10	0.5	2000
# 3	0.9614	0.695333	gini	10	0.8	2000
# 2	0.9622	0.693333	gini	10	0.8	500
# 0	0.9668	0.692000	gini	10	0.5	500

### Evaluation confusion matrix on Top 2 models (based on accuracy) with depth of 10 and 5

### Model 1: (4 hyper-parameters tuned: n_estimators = 2000, criterion = gini , max_features = 0.5, max_depth = 5), depth of 5 to limit overfitting

In [30]:
# Create Random Forest pipeline
pipe_rf1 = Pipeline(
    [
        ("scaler", StandardScaler()),  # with standardization StandardScaler()
        (
            "PCA",
            PCA(n_components=200),
        ),  # 200 components to explain 95% of the variance (see first part of this notebook)
        (
            "rf",
            RandomForestClassifier(
                n_jobs=-1,
                n_estimators=2000,
                criterion="gini",
                max_features=0.5,
                warm_start=True,
                max_depth=5,
                random_state=0,
            ),
        ),
    ]
)

In [31]:
# Fit a Decision Tree classifier
model_rf1 = pipe_rf1.fit(X_tr_3rd, y_tr_3rd)

In [32]:
# Make prediction on X_valid dataset
y_pred_rf1 = pipe_rf1.predict(X_valid2)

In [33]:
# Confusions report
target_names = ["class 0", "class 1", "class 2"]
print(classification_report(y_valid2, y_pred_rf1, target_names=target_names))

              precision    recall  f1-score   support

     class 0       0.78      0.81      0.80       486
     class 1       0.57      0.48      0.52       467
     class 2       0.73      0.80      0.76       547

    accuracy                           0.70      1500
   macro avg       0.69      0.70      0.69      1500
weighted avg       0.70      0.70      0.70      1500



### Model 1 based on larger set of data (train 15 000, test 4 000) - (4 hyper-parameters tuned: n_estimators = 2000, criterion = gini, max_features = 0.5, max_depth = 5), depth of 5 to limit overfitting

In [34]:
# Create Random Forest pipeline
pipe_rf1b = Pipeline(
    [
        ("scaler", StandardScaler()),  # with standardization StandardScaler()
        (
            "PCA",
            PCA(n_components=200),
        ),  # 200 components to explain 95% of the variance (see first part of this notebook)
        (
            "rf",
            RandomForestClassifier(
                n_jobs=-1,
                n_estimators=2000,
                criterion="gini",
                max_features=0.5,
                warm_start=True,
                max_depth=5,
                random_state=0,
            ),
        ),
    ]
)

In [35]:
# Fit a Decision Tree classifier
model_rf1b = pipe_rf1b.fit(X_tr_2nd, y_tr_2nd)

In [36]:
y_pred_rf1b = pipe_rf1b.predict(X_valid1)

In [37]:
# Confusions report
target_names = ["class 0", "class 1", "class 2"]
print(classification_report(y_valid1, y_pred_rf1b, target_names=target_names))

              precision    recall  f1-score   support

     class 0       0.77      0.79      0.78      1240
     class 1       0.59      0.52      0.55      1328
     class 2       0.74      0.80      0.77      1432

    accuracy                           0.70      4000
   macro avg       0.70      0.70      0.70      4000
weighted avg       0.70      0.70      0.70      4000



### Model 5 (6 hyper-parameters tuned: n_estimators = 2000, min_samples_leaf = 4, bootstrap = False, criterion = gini, max_features = 0.5, max_depth = 10), model with the highest accuracy (but with some overfit)

In [38]:
# Create Random Forest pipeline
pipe_rf2 = Pipeline(
    [
        ("scaler", StandardScaler()),  # with standardization StandardScaler()
        (
            "PCA",
            PCA(n_components=200),
        ),  # 200 components to explain 95% of the variance (see first part of this notebook)
        (
            "rf",
            RandomForestClassifier(
                n_jobs=-1,
                n_estimators=2000,
                criterion="gini",
                min_samples_leaf=4,
                bootstrap=False,
                max_features=0.5,
                warm_start=True,
                max_depth=10,
                random_state=0,
            ),
        ),
    ]
)

In [39]:
# Fit a Decision Tree classifier
model_rf2 = pipe_rf2.fit(X_tr_3rd, y_tr_3rd)

In [40]:
# Make prediction on X_valid dataset
y_pred_rf2 = pipe_rf2.predict(X_valid2)

In [41]:
# Confusions report
target_names = ["class 0", "class 1", "class 2"]
print(classification_report(y_valid2, y_pred_rf2, target_names=target_names))

              precision    recall  f1-score   support

     class 0       0.83      0.78      0.81       486
     class 1       0.60      0.61      0.61       467
     class 2       0.77      0.81      0.79       547

    accuracy                           0.74      1500
   macro avg       0.74      0.73      0.73      1500
weighted avg       0.74      0.74      0.74      1500



### Conclusions
> * The Best model candidate for Random Forest classifier is the **model 4** Random Forest, n_estimators: 2 000, max_features = 0.5, max_depth = 5
* This model presents acceptable levels of accuracy (0.99 on train and 0.71 on valid). However, the difference between the 2 levels of accuracy indicates important overfitting
* We will perform a cross-validation with 5 folds in order to evaluate if we could limit the overfitting despite the small number of datapoints (due to limited computer resources)

## Run selected random Forest model on Train/Test dataset using cross-validation with 5 kfolds
> * It appears that (probably due to the small number of training dataset), models tends to overfit and do not generalize well. We will try a cross-validation approach with 5 folds in order to see if we could minimize the overfit despite the small number of datapoints (model 5)

### Create a cross validation object

In [42]:
# Create Random Forest pipeline
pipe_rf_grid = Pipeline(
    [
        ("scaler", StandardScaler()),  # with standardization StandardScaler()
        ("PCA", PCA()),
        ("rf", RandomForestClassifier(n_jobs=-1, random_state=0)),
    ]
)

In [43]:
# Create cross-validation object
grid_rf = GridSearchCV(
    pipe_rf_grid,
    [
        {
            "PCA__n_components": [
                200
            ],  # nb of components explaining 95% of the variance
            "rf__n_estimators": [2000],  # of decision trees
            "rf__criterion": ["gini"],  # quality of split
            "rf__max_features": [
                0.5
            ],  # % of features vs total number of features for looking at the best split
            "rf__max_depth": [10],  # depth of trees
        }
    ],
    cv=5,
)

In [44]:
# Fit estimator
grid_rf.fit(X_tr_3rd, y_tr_3rd)

GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=Pipeline(memory=None,
                                steps=[('scaler',
                                        StandardScaler(copy=True,
                                                       with_mean=True,
                                                       with_std=True)),
                                       ('PCA',
                                        PCA(copy=True, iterated_power='auto',
                                            n_components=None,
                                            random_state=None,
                                            svd_solver='auto', tol=0.0,
                                            whiten=False)),
                                       ('rf',
                                        RandomForestClassifier(bootstrap=True,
                                                               class_weight=None,
                                                      

In [45]:
# Get the results with "cv_results_"
grid_rf.cv_results_.keys()

dict_keys(['mean_fit_time', 'std_fit_time', 'mean_score_time', 'std_score_time', 'param_PCA__n_components', 'param_rf__criterion', 'param_rf__max_depth', 'param_rf__max_features', 'param_rf__n_estimators', 'params', 'split0_test_score', 'split1_test_score', 'split2_test_score', 'split3_test_score', 'split4_test_score', 'mean_test_score', 'std_test_score', 'rank_test_score'])

In [46]:
# Define the hyperparameters to collect into the dataframe as arrays
pca =  grid_rf.cv_results_["param_PCA__n_components"]
n_estimators = grid_rf.cv_results_["param_rf__n_estimators"]
criterion = grid_rf.cv_results_["param_rf__criterion"]
mean_te = grid_rf.cv_results_["mean_test_score"]
std_test_score =  grid_rf.cv_results_["std_test_score"]

In [50]:
# Create the dataframe using from_dict (as from_items is deprecated) and OrderedDict to keep the order
df_rf_grid = pd.DataFrame.from_dict(OrderedDict(zip(['pca', 'n_estimators', 'criterion', 'mean_te', 'std_test_score'], [pca, n_estimators, criterion, mean_te, std_test_score])))
df_rf_grid.sort_values(by='mean_te', ascending=False)

,pca,n_estimators,criterion,mean_te,std_test_score
0,200,2000,gini,0.7012,0.012698


### Conclusions on Random Forest with cross-validation
> * It is not really conclusive as the 2nd best model (with the same set of hyper-parameter tuning) is performing less (accuracy on test 0.8262) than the same one with train/valid dataset (acc. 0.85)
* As a consequence, we will select the standard approach which provides slightly better results and being less computational resource demanding

### Save results for later visualization and overall selection - Best Random Forest Model with 4 hyper-parameters ( n_estimators = 2000, criterion = 'gini', max_features = 0.5, max_depth = 5, )

In [ ]:
rf_acc = 0.71
c1_rf_f1 = 0.80
c2_rf_f1 = 0.53
c3_rf_f1 = 0.76

%store rf_acc
%store c1_rf_f1
%store c2_rf_f1
%store c3_rf_f1